Table 4.12.2: Number of children living with individuals starting treatment 2017-18

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.12.2']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
observations = tab.excel_ref('B6').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<C7 4020.0>, <E6 20904.0>, <B6 10328.0>, <E7 11967.0>, <D7 4009.0>, <D6 7453.0>, <F8 1.80162544445747>, <C6 7424.0>, <D8 1.85906709902719>, <F6 46109.0>, <B7 5597.0>, <E8 1.74680371020306>, <C8 1.84676616915423>, <B8 1.84527425406468>, <F7 25593.0>}

In [4]:
child = tab.excel_ref('A6').expand(DOWN).is_not_blank() - tab.excel_ref('A19').expand(DOWN)
child

{<A7 'Number of individuals living with children'>, <A10 '*Percentages may equal 0% or not sum to 100% due to rounding'>, <A6 'Number of children'>, <A8 'Average number of children living with people in treatment'>}

In [5]:
Clients = tab.excel_ref('B4').expand(RIGHT).is_not_blank()
Clients

{<E4 'Alcohol only'>, <B4 'Opiate'>, <C4 'Non-opiate only'>, <D4 'Non-opiate and Alcohol'>, <F4 'Total'>}

In [6]:
Dimensions = [
            HDim(child,'Category',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')            
            ]

In [7]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [8]:
new_table = c1.topandas()
new_table

,OBS,Category,Clients in treatment,Measure Type,Unit
0,10328.000000,Number of children,Opiate,Count,People
1,7424.000000,Number of children,Non-opiate only,Count,People
2,7453.000000,Number of children,Non-opiate and Alcohol,Count,People
3,20904.000000,Number of children,Alcohol only,Count,People
4,46109.000000,Number of children,Total,Count,People
5,5597.000000,Number of individuals living with children,Opiate,Count,People
6,4020.000000,Number of individuals living with children,Non-opiate only,Count,People
7,4009.000000,Number of individuals living with children,Non-opiate and Alcohol,Count,People
8,11967.000000,Number of individuals living with children,Alcohol only,Count,People
9,25593.000000,Number of individuals living with children,Total,Count,People


In [9]:
new_table = new_table[new_table['OBS'] != 0 ]

In [10]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [11]:
new_table.head()

,Value,Category,Clients in treatment,Measure Type,Unit
0,10328.0,Number of children,Opiate,Count,People
1,7424.0,Number of children,Non-opiate only,Count,People
2,7453.0,Number of children,Non-opiate and Alcohol,Count,People
3,20904.0,Number of children,Alcohol only,Count,People
4,46109.0,Number of children,Total,Count,People


In [12]:
new_table.tail()

,Value,Category,Clients in treatment,Measure Type,Unit
10,1.845274,Average number of children living with people ...,Opiate,Count,People
11,1.846766,Average number of children living with people ...,Non-opiate only,Count,People
12,1.859067,Average number of children living with people ...,Non-opiate and Alcohol,Count,People
13,1.746804,Average number of children living with people ...,Alcohol only,Count,People
14,1.801625,Average number of children living with people ...,Total,Count,People


In [13]:
new_table.dtypes

Value                   float64
Category                 object
Clients in treatment     object
Measure Type             object
Unit                     object
dtype: object

In [14]:
new_table['Value'] = new_table['Value'].astype(str)

In [15]:
new_table.head(3)

,Value,Category,Clients in treatment,Measure Type,Unit
0,10328.0,Number of children,Opiate,Count,People
1,7424.0,Number of children,Non-opiate only,Count,People
2,7453.0,Number of children,Non-opiate and Alcohol,Count,People


In [16]:
new_table['Category'] = new_table['Category'].map(
    lambda x: {
        'Number of children' : 'Number of children living with people in treatment',
        'Number of individuals living with children' : 'Number of clients living with children'
        }.get(x, x))

In [17]:
new_table.head(3)

,Value,Category,Clients in treatment,Measure Type,Unit
0,10328.0,Number of children living with people in treat...,Opiate,Count,People
1,7424.0,Number of children living with people in treat...,Non-opiate only,Count,People
2,7453.0,Number of children living with people in treat...,Non-opiate and Alcohol,Count,People


In [18]:
new_table['Clients in treatment'] = 'New clients/' + new_table['Clients in treatment']

In [19]:
new_table['Period'] = '2017-18'
new_table.columns = ['Basis of treatment' if x=='Category' else x for x in new_table.columns]
new_table['Substance'] = 'All'
new_table = new_table[['Period','Basis of treatment','Substance','Clients in treatment','Measure Type','Value','Unit']]

In [20]:
new_table.head()

,Period,Basis of treatment,Substance,Clients in treatment,Measure Type,Value,Unit
0,2017-18,Number of children living with people in treat...,All,New clients/Opiate,Count,10328.0,People
1,2017-18,Number of children living with people in treat...,All,New clients/Non-opiate only,Count,7424.0,People
2,2017-18,Number of children living with people in treat...,All,New clients/Non-opiate and Alcohol,Count,7453.0,People
3,2017-18,Number of children living with people in treat...,All,New clients/Alcohol only,Count,20904.0,People
4,2017-18,Number of children living with people in treat...,All,New clients/Total,Count,46109.0,People
